In [1]:
import os
import pandas as pd

pred_path = '../n2c2-data/output'
gold_path = '../n2c2-data/test'

In [21]:
with open('../resources/ncbi-wiki-abbreviations.txt') as f:
    ncbi = [x.strip() for x in f.readlines()]
with open('../resources/semeval-wiki-abbreviations.txt') as f:
    semeval = [x.strip() for x in f.readlines()]
spell = list(set(ncbi+semeval))
spell.sort()

with open('../resources/abbreviations.txt','w+') as f:
    for s in spell:
        f.write(s+'\n')

In [3]:
files = os.listdir(pred_path)
for file in files:
    pred = pd.read_table(f'{pred_path}/{file}',sep='\|\|',header=None, names=['file_id','ix','mention','cui'])
    gold = pd.read_table(f'{gold_path}/{file}',sep='\|\|',header=None, names=['file_id','ix','mention','cui'])
    break
pred

C:\Users\etfrench\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\etfrench\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  after removing the cwd from sys.path.


,file_id,ix,mention,cui
0,34,527|531,pain,D010146
1,34,534|542,headache,C0018681
2,34,545|556,temperature,C0204688
3,34,647|651,pain,D010146
4,34,680|682,rr,C1443397
...,...,...,...,...
1906,34,3921|3924,rle,C0230415
1907,34,3893|3913,physical examination,C0031809
1908,34,4700|4711,medications,C0013216
1909,34,4731|4737,motrin,C0699203


In [27]:
df = pd.read_csv('test4.txt',sep='\t',header=None)
df.to_csv('test4_pipe.txt',sep='|',header=None,index=None)

In [18]:
df[1] = df[1].replace('"','')
df.iloc[57]

0                                                   58
1    Fast thinker. \nVery efficient to work. Gets a...
Name: 57, dtype: object